In [37]:
import pickle
import numpy as np

# read my pickle
data = pickle.load(open("individuals_friendships_utilities.p","rb"))
characteristics, connectivity, utilities = data

X_list = pickle.load(open("X_list.pkl", "rb"))

In [10]:
for i in range(len(X_list)):
    print(X_list[i].shape)
    print(X_list[i], '\n')

(71, 6)
[[ 1.  1. 12.  1.  0.  0.]
 [ 1.  1.  7.  1.  0.  0.]
 [ 2.  1.  9.  1.  0.  0.]
 [ 1.  5.  7.  0.  0.  1.]
 [ 1.  3. 10.  0.  1.  0.]
 [ 2.  1.  9.  1.  0.  0.]
 [ 2.  1.  8.  1.  0.  0.]
 [ 1.  1. 10.  1.  0.  0.]
 [ 1.  1. 11.  1.  0.  0.]
 [ 2.  1.  7.  1.  0.  0.]
 [ 2.  1.  8.  1.  0.  0.]
 [ 2.  1. 11.  1.  0.  0.]
 [ 2.  5.  8.  0.  0.  1.]
 [ 2.  1. 10.  1.  0.  0.]
 [ 2.  1.  8.  1.  0.  0.]
 [ 1.  1.  8.  1.  0.  0.]
 [ 2.  1.  8.  1.  0.  0.]
 [ 1.  1. 10.  1.  0.  0.]
 [ 1.  1.  9.  1.  0.  0.]
 [ 2.  1.  7.  1.  0.  0.]
 [ 1.  1.  8.  1.  0.  0.]
 [ 1.  1.  8.  1.  0.  0.]
 [ 2.  1. 11.  1.  0.  0.]
 [ 1.  1.  8.  1.  0.  0.]
 [ 1.  1.  9.  1.  0.  0.]
 [ 1.  5.  9.  0.  0.  1.]
 [ 2.  1.  7.  1.  0.  0.]
 [ 2.  1. 11.  1.  0.  0.]
 [ 2.  1.  9.  1.  0.  0.]
 [ 2.  1. 10.  1.  0.  0.]
 [ 2.  1.  8.  1.  0.  0.]
 [ 2.  5. 11.  0.  0.  1.]
 [ 1.  1.  7.  1.  0.  0.]
 [ 2.  1.  9.  1.  0.  0.]
 [ 2.  1. 11.  1.  0.  0.]
 [ 1.  1. 11.  1.  0.  0.]
 [ 1.  1. 11.  1.  0

In [41]:
def ana_dyads(connectivity):
    """
        Calculate proportion symmetric dyads (PTCMUT), asymmetric dyads (PTCASY) and mutuality index (RHO2).
    """
    nodes = connectivity.shape[0]
    total_d = np.count_nonzero(connectivity == 1)
    mutual_d = 0
    asym_d = 0
    out_degrees = []
    
    # count out_degree connections per individual and mutual and asymmetrix connections whole network
    for i in range(connectivity.shape[0]):
        out_degree = 0
        for j in range(connectivity.shape[1]):
            if connectivity[i][j] == 1:
                out_degree += 1
                if connectivity[j][i] == 1:
                    mutual_d += 1
                elif connectivity[j][i] != 1:
                    asym_d += 1
        out_degrees.append(out_degree)
    
    # calculate proportion symmetric dyads (PTCMUT) and asymmetric dyads (PTCASY)
    PTCMUT = mutual_d / total_d
    PTCASY = asym_d / total_d
    
    # calculate mutuality index (RHO2) (according to Katz and Powell’s (1955))
    sum_out_degrees = sum(out_degrees)
    mean_out_degrees = sum_out_degrees / len(out_degrees)
    sum_squares_out = 0
    for i in range(len(out_degrees)):
        sum_squares_out += (mean_out_degrees - out_degrees[i])**2
    RHO2 = (2*(nodes - 1)**2 * mutual_d - sum_out_degrees**2 + sum_squares_out) / (sum_out_degrees*(nodes - 1)**2 - sum_out_degrees**2 + sum_squares_out)

    return PTCMUT, PTCASY, RHO2

ana_dyads(connectivity)

(0.5487804878048781, 0.45121951219512196, 1.1027171109002092)